In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<font size=6>**Introduction to Bayesian Statistics**</font>


# 1. The problem: Counting photons...

Typically, an X-ray imaging observation gives a collection of **events**: coordinates and energies of **individual photons** detected by the telescope. Essentially, we count photons, and this is why we usually refer to the photons as **counts**, while the **count rate** informs us about the number of photons per second. In general, the count rate scales with the flux of the field (or source) in the energy band of the telescope during the observation.

<table><tr>
<td> <img src="images/NGC1482_opt.jpg" alt="Drawing" style="height:400px;"/> </td>
<td> <img src="images/NGC1482.png" alt="Drawing" style="height: 400px;"/> </td>
</tr></table>


Using **source detection algorithms** we group photons together and generate a list of sources and the photons we got from each - namely, the *source region*. For a given source, the integrated energy of the photons from its source region gives us an estimate on the energy collected from the telescope. Of course, the response of the detector and absorption effects should be accounted for. Having many counts allows us to fit for the spectrum of the source. Modeling the detector properties, the intergalactic absorption and the spectra allows us to measure the *bolometric* flux of the source - the flux as if we could observe the source with a perfect detector. Provided we know the distance of the source we can convert from flux to luminosity... an intrinsic property of the source (which is what we usually care for)!

## 1.1. Estimate the flux of a source

Using the Chandra X-ray Observatory, we found a source that emitted 5 counts in an observation of 1ks of the galaxy NGC 1482. What is the luminosity of the source?

* Using a typical model for the spectrum of an X-ray source (power law with index=1.7), and Galactic absorption $N_{\mathrm{H}}=4.07\times10^{20}\,\mathrm{cm}^{-2}$ (corresponding to the direction of NGC 1482 galaxy), as well as the characteristics of the telescope, we found that: 1 count/s corresponds to a flux of $2.851\times 10^{-11}$ erg/s/cm$^2$.
* The distance of NGC 1482 is estimated to be 19.6 Mpc.

In [ ]:
exposure_time_in_seconds = 1.0e3   # from the duration of the observation
distance_in_Mpc = 19.6             # from literature
counts_to_flux_factor = 2.851e-11  # from X-ray observers' tools (e.g., Chandra PIMMS tool)
counts = 5                         # from source detection


counts_per_second = counts / exposure_time_in_seconds
print(f"Count rate : {counts_per_second:10.3g} count/s")

source_flux = counts_to_flux_factor * counts_per_second
print(f"Source flux: {source_flux:10.3g} erg/s/cm^2")

flux_to_luminosity = 1.19e50 * distance_in_Mpc ** 2.0
source_luminosity = source_flux * flux_to_luminosity
print(f"Luminosity : {source_luminosity:10.3g} erg/s")

## 1.2. What is the uncertainty on this luminosity?

When counting things that are not correlated together, we expect a **Poisson distribution**. If an X-ray source emits on average $\lambda$ photons that can be detected by our telescope during the observation (accounting for absorption, instrumentation, etc.) then the probability to detect any number of photons is given by the **probability mass function** of the Poisson-distributed random variable $X$ with parameter $\lambda$:

$$\large \mathrm{Pois}(k; \lambda) = \dfrac{ \lambda^k e^{-\lambda} }{ k! } $$

Conveniently, the parameter $\lambda$ is also the **expected value** or the mean of the Poisson distribution:

$$\large \mu = \mathrm{E}[X] = \sum_{k=0}^{\infty} k \times \mathrm{Pois}(k; \lambda) = \cdots = \lambda $$

The standard deviation of the distribution, which is a measure of the *spread of the random samples from the expected value* is the square root of the variance which is equal to $\lambda$:

$$\large \sigma = \sqrt{\mathrm{Var}[X]} = \sqrt{ \mathrm{E}\left[\left(X-\mathrm{E}[X]\right)^2\right] } = \cdots = \sqrt{\lambda} $$

### What most of us have in mind when thinking of uncertainties, sigmas, etc.
To put things in perspective, in the commonly-appeared/used normal (or Gaussian) distribution, **within one standard deviation** from the expected value lies the $\sim68\%$ of the samples:

<img src="images/normal_dist.png" alt="Drawing" style="height:400px;"/>

### Let's see the Poisson distribution with $\lambda = 5$...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

lam = 5.0
k = np.arange(0, 20)
Pk = st.poisson(lam).pmf(k)

in_CI = (k >= lam-lam**0.5) & (k <= lam+lam**0.5)
fraction_in_CI = f"{100.0 * sum(Pk[in_CI]):.1f}%"

real_interval = st.poisson(lam).interval(0.68)

plt.figure()

plt.bar(k[in_CI], Pk[in_CI], color="r", label=fraction_in_CI+r" in $\left[5-\sqrt{5}, 5+\sqrt{5}\right]$")
plt.bar(k[~in_CI], Pk[~in_CI], color="b")

plt.axvline(real_interval[0], color="k", ls=":", lw=2, label="68% CI")
plt.axvline(real_interval[1], color="k", ls=":", lw=2)

plt.ylabel("Probability mass (unitless)")
plt.xlabel("Number of events, counts, sources, etc.")
plt.legend(loc="upper right")
plt.show()

### What is the uncertainty on the source luminosity according the Poisson-nature of the counts?

In [ ]:
counts_uncertainty = counts ** 0.5
luminosity_uncertainty = flux_to_luminosity * counts_to_flux_factor * counts_uncertainty / exposure_time_in_seconds
print(f"Uncertainty on luminosity: {luminosity_uncertainty:.3g} erg/s")
print(f"We may report: ({source_luminosity/1.e39:.3g} +/- {luminosity_uncertainty/1.e39:.3g}) x 10^39 erg/s")

## 1.3. Are we using the Poisson nature of the counts the correct way?

The uncertainy is calculated as the square root of the mean value of the Poisson distribution. **But we don't know the expected value!** We have only observed one sample from the Poisson distribution.

> We can get 5 counts if the mean value is 5, but also in the case of any different mean value!

## Exercise 1: Pick three different natural numbers for the  mean of the Poisson (including 5)

In [ ]:
xaxis_maximum = 12
poisson_means = [..., 5, ...]

plt.figure()
for mean_value in poisson_means:
    k = np.arange(int(xaxis_maximum)+1)
    Pk = st.poisson(mean_value).pmf(k)
    plt.plot(k, Pk, "o:", label=f"$\lambda$ = {mean_value}")
plt.axvline(5, color="0.5", label="Observed")
plt.ylabel("Probability mass")
plt.xlabel("Number of counts")
plt.ylim(ymax=0.25)
plt.legend(loc="upper right")
plt.title("Probability of 5 counts with different $\lambda$ parameters")
plt.show()

## But isn't the observation a good estimate on the mean value?

Someone could argue that the peak of the distribution is the most probable value. Therefore, measuring one sample provides some information on the mean value, and consequently on the standard deviation (since the two are related in the Poisson distribution)

### However...

> Many different parameters in a distribution may result in the same peak value!

The peak of Pois(5) is both at 4 and 5! So, if we detected 4 counts, should we have inferred that $\lambda=5$ instead of $\lambda=4$?

Similarily, the peak of Pois(6) is both at 5 and 6!

In [ ]:
xaxis_maximum = 12
poisson_means = [4, 5, 6]

plt.figure()
for mean_value in poisson_means:
    k = np.arange(int(xaxis_maximum)+1)
    Pk = st.poisson(mean_value).pmf(k)
    plt.plot(k, Pk, "o:", label=f"$\lambda$ = {mean_value}")
plt.axvline(5, color="0.5", label="Observed")
plt.ylabel("Probability")
plt.xlabel("Number of counts")
plt.ylim(ymax=0.25)
plt.legend(loc="upper right")
plt.title("Probability of 5 counts with 'nearby' $\lambda$ parameters")
plt.show()

## Exercise 2: $\lambda$ is a real number!

Pick a few values close to 5 to see the shape of the Poisson distribution!

In [ ]:
xaxis_maximum = 12
poisson_means = [..., 5.0, ...]

plt.figure()
for mean_value in poisson_means:
    k = np.arange(int(xaxis_maximum)+1)
    Pk = st.poisson(mean_value).pmf(k)
    plt.plot(k, Pk, "o:", label=f"$\lambda$ = {mean_value}")
plt.axvline(5, color="0.5", label="Observed")
plt.ylabel("Probability")
plt.xlabel("Number of counts")
plt.ylim(ymax=0.25)
plt.legend(loc="upper right")
plt.title("Probability of 5 counts with $\lambda$ parameters close to 5")
plt.show()

# 2. The Bayesian probability

It's all about our interpretation of probability, and how we represent knowledge. 

## The classic (frequentist) and the *Bayesian way*?

> A frequentist assigns probabilities to data - parameters are fixed. The probability is a *frequency* of data outcomes.

We did the above analysis assuming the source counts are equal to the detected ones, and assigned probabilities to different data outcomes. Our inference on the source luminosity and its uncertainty is telling us that if we got a good estimate from the observation, *repeating the experiment many times* we will find the above variation in the luminosity.

**However, we have no indication that the source will continue shining! The "repetition" part of the frequentist interpretation is often unrealistic!**

> A Bayesian assigns probabilities to hypotheses. The probability is a *degree of belief* in a value of a parameter.

- The data is not a random variable. We made a measurement and we know it with absolute certainty. Instead, we want to understand the physical processes producing the data we got.
- Knowing the result from repeating the experiment is very specific: maybe we care about what was the luminosity at that given period of time. This is not a repeatable situation!

![image from Sivia](images/Sivia_Logic.png)

The intrinsic source luminosity (and therefore the expected counts) is unknown and we want to infer it from the data. Therefore we ask: what is the probability that our source has an average count $\lambda$ given the data (detected number of counts) for all possible values of $\lambda$ (or hypotheses)?

$$\large P(\lambda | k) = \mathrm{?}$$

## Probability calculus (Freq. and Bays. alike)

How do we begin to calculate the above probability?

When we are dealing with probabilties, there are useful notations and rules:

$ P(A) $ is a number between 0 and 1 denoting our degree of belief that a given event/hypothesis $A$ is true.

$$\large P(\text{tails}) = 0.5 $$

$ P(\bar{A})$ is the probability that $A$ is not true, and it is equal to $1-P(A)$.

$$\large P(\text{heads}) = P(\text{not tails}) = 1 - P(\text{tails}) = 0.5 $$

$ P(A, B) $ is the **joint** probability that $A$ and $B$ are both true. Of course $P(A, B) = P(B, A)$

$$\large P(\text{we see tails}) = P(\text{the coin was tossed}, \text{tails}) $$

$ P(A | B)$ is the conditional probability: $A$ to be true if $B$ is true, or **A given B**.

$$\large P(\text{tails} | \text{the coin was tossed}) = 0.5 $$
$$\large P(\text{tails} | \text{the coin was not tossed}) = 0 $$

$P(A, B) = P(A | B) P(B) $ is the chain rule which allows us to factor joint probabilities or calculate conditional probabilities.

We throw two coins...
$$\large P(\text{heads from 1st}, \text{heads from 2nd}) = P(\text{heads from 1st} | \text{heads from 2nd}) \times P(\text{heads from 2nd}) = 0.5 \times 0.5 = 0.25 $$
(where we assumed that the tossings are independent).

We have two coins. We throw the first, and if the result is tails, we are allowed to throw the next one.

$$\large P(\text{heads from 1st}, \text{heads from 2nd}) = P(\text{heads from 2nd}, \text{heads from 1st}) = P(\text{heads from 2nd} | \text{heads from 1st}) \times P(\text{heads from 1st}) = 0 \times 0.5 = 0 $$

## Bayes' theorem... law... rule...

Applying the chain rule we have two ways to write the probability of two events:

$$\large P(A, B) = P(A | B) P(B) $$
$$\large P(B, A) = P(B | A) P(A) $$

By equalizing the right-hand sides we get:

$$\large P(A | B) = \dfrac{ P(B|A) P(A) }{ P(B) } $$

So in data analysis we can characterize the belief in a hypothesis using data in the following way:

$$\large P(\text{hypothesis} | \text{data}, I ) = \dfrac{ P(\text{data | hypothesis}, I) P(\text{hypothesis} | I)}{ P(\text{data} | I) } $$

The symbol $I$ denotes the **background information**: all relevant knowledge we have about the problem we are solving but are not part of the data.

For example, if we are testing whether a die is fair, the data are a sequence of outcomes (e.g., 1, 6, 3, 5, 3, 1), the hypothesis is **it's a fair die**, and $I$ = $\big\{$ all dice have 6 sides, a fair die has equiprobable sides, the Moon is not made of cheese, $\cdots \big\}$.

For brevity, **we usually omit writing the $I$** in the equations, but **it's always there**... somewhere in the background!

All the terms of this equation, the famous Bayes' rule, have names:

* **Posterior**: $P(\text{hypothesis} | \text{data}, I)$ is the degree of belief we have on the hypothesis after (a posteriori) looking at the data
* **Likelihood**: $P(\text{data | hypothesis}, I)$ is the likelihood of collecting the data at hand, given that the hypothesis is true (what a frequentist would focus on...)
* **Prior**: $P(\text{hypothesis} | I)$ is the degree of belief in the hypothesis before looking at the data (a priori). E.g., from previous studies, mathematical or physical constraints, ...
* **Evidence**: $P(\text{data} | I)$ is the probability of getting the data independently of whether the hypothesis is true or false. We almost never know the evidence and in most problems its value is not affecting the analysis!

> In Bayesian Analysis we assign degrees of belief to hypotheses, which we "update" using experimental data

## 3. The Bayesian approach to our photon-counting problem

$$\large 
P(\lambda | k) = 
\dfrac{ \large P(k | \lambda) P(\lambda) }{ P(k) }
$$

*(for brevitiy we omitted the background information $I$)*

## 3.1. Constructing the likelihood

The likelihood is expressed in terms of the PDF of the Poisson with expected value the unknown parameter $\lambda$:

$$\large k \sim \text{Pois}(\lambda)$$

$$\large
P(k | \lambda) = \text{Pois}(k; \lambda) = \dfrac{\lambda^k e^{-\lambda}}{k!}
$$

*(usually words like Pois, Norm, Gamma, etc. describe distributions. Using the Dist(x; p) notation we are referring to the probability mass function (discrete distirbutions) or the probability density function (continuous distributions) with parameters $p$ evaluated at $x$)*

## Exercise 3: Pick the correct function to plot the likelihood

You can consult the [scipy.stats.poisson documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html) to find the probability mass function of the Poisson distribution.

In [ ]:
lambdas = np.linspace(0.0, 20, 200)
k = counts

likelihoods = st.poisson. ...(k, lambdas)


plt.figure()
plt.plot(lambdas, likelihoods, "k-")
plt.xlabel("$\lambda$")
plt.ylabel("Likelihood $P(5; \lambda)$")
plt.show()

This is not a Poisson distribution, since the "random variable" is now $\lambda$ which is continuous (Poisson is a *discrete* distribution)! In reality this is the **Gamma distribution** with shape parameter $\kappa=k+1$ and scale parameter $\theta=1$, or simply $\text{Gamma}(k+1, 1)$.

$$\large
P(k | \lambda) = \text{Pois}(k; \lambda) = \dfrac{\lambda^k e^{-\lambda}}{k!} = \dfrac{\lambda^{(k+1)-1} e^{-\lambda}}{\Gamma(k+1)} = \text{Gamma}(\lambda; k+1, 1)
$$

## 3.2. Constructing the prior

What about the prior? If we have no previous information about the source intensity, then we could be completely agnostic and consider all possible values as equal:

$$\large  P(\lambda) = \text{a positive constant}, \quad \lambda \in \mathbb{R} $$ 

Constant priors of an infinite or finite range are called **uniform priors**. However this is not an actual probability density function, since it's integration over the whole range $\lambda \in \left(-\infty, \infty\right)$ would lead to infinity. Instead, we could take a safe, huge, but finite range. At the end of the day, it doesn't make any difference, so we often use these **improper priors**.

In our case we can do better by requiring the $\lambda$ is positive since our source luminosity cannot be negative!

$$\large  P(\lambda) = A \begin{cases} 1, \quad \lambda \geq 0 \\ 0, \quad \lambda < 0 \end{cases} $$

Note that this is still an **imporper prior**. We can even omit the normalization factor $A$ since we cannot rescale an improper prior. In practice, we work with proportionalities when the scale is unknown (or irrelevant): $P(\lambda) \propto \cdots$.

But to do things "properly"... we can assume a uniform distribution. Maybe previous studies showed that point sources in galaxies have an upper limit in the luminosity, which in our observation corresponds to 10 counts.

$$\large  P(\lambda) = A \begin{cases} 1, \quad \lambda \in [0, 10] \\ 0, \quad \text{elsewhere} \end{cases} $$

Normalizing the prior results in:

$$\large \int_{-\infty}^{\infty} P(\lambda) d\lambda = 1 \Rightarrow \int_{0}^{10} A d\lambda = 1 \Rightarrow A = \frac{1}{10} $$

## 3.3. The evidence

The term $ P(k)$ is called *evidence*, or marginal likelihood. It describes the probability of getting the data given the background information, but regardless of the parameters of the models (or the hypotheses). In the theory example, this is:

$$\large P(\text{data}) = P(\text{data | hypothesis}) P(\text{hypothesis}) + P(\text{data} | \overline{\text{hypothesis}}) P(\overline{\text{hypothesis}}) $$

Therefore, it is the summation of the likelihood of the data considering all possible outcomes of the experiment. In the case that we are calculating the posterior for a model parameter, say, the number of lines in a spectrum we would have a posterior looking like this:

$$\large P(\text{num. of lines | spectrum}) = \dfrac{ P(\text{spectrum | num. of lines}) P(\text{num. of lines}) }
   { P(\text{spectrum}) } $$
   
with the evidence being:

$$\large P(\text{spectrum}) = P(\text{spectrum | no lines}) P(\text{no lines}) + P(\text{spectrum | one line}) P(\text{one line}) + \cdots = \sum_{n=0}^{\infty} P(\text{spectrum | num. of lines = n}) P(\text{num. of lines = n}) $$

Notice, that we are practically integrating, or **marginalizing** the numerator of the Bayes formula, hence, normalizing the probabilities. Notice, however, that since the evidence does not depend on the hypothesis, when calculating the **relative** probabilities of different number of lines, the evidence is **cancelled out**. For this reason, **in most problems we can ignore this term, and only if necessary, we calculate it to normalize the posterior.** When selecting between hypothesis, or different models, the evidence may be imporant.

In our source photon problem the evidence is
$$\large P(k) = \int_{-\infty}^{\infty} P(k | \lambda) P(\lambda) d\lambda $$
which just normalizes the numerator, and we can ignore it.


## 3.4. How the posterior looks now...

$$\large 
P(\lambda | k) = 
    \begin{cases}
        \dfrac{\lambda^k e^{-\lambda}}{k!} \times \frac{1}{10} \times \dfrac{1}{\text{evidence}} &,\quad \lambda \in [0, 10]
\\
        0 & ,\quad \text{elsewhere}
    \end{cases}
$$

Since we ignore the normalization from the evidence, it doesn't make sence to keep the normalization of the prior. Any uniform prior can be neglected, but we should make sure we are respecting the bounds of the priors (if any)! Therefore,

$$\large 
P(\lambda | k) \propto 
    \begin{cases}
        \dfrac{\lambda^k e^{-\lambda}}{k!} &,\quad \lambda \in [0, 10]
\\
        0 &,\quad \text{elsewhere}
    \end{cases}
$$


### A way of working...

The extended bibliography in classical statistics, the standardization of its methods, and the numerous implementations in software may be powerful and useful but often obscure the assumptions and caveats we are trading in. During our Bayesian analysis, writing down the equations and deriving the final formula, made us consider the nature of the data, our previous knowledge, assumptions, background information, etc.

### A way of thinking...

Notice that this is very similar to the likelihood a frequentist would use. The **practical difference** is that we are now estimating $\lambda$, taking into account prior information. The **conceptual difference** is that we assigned a probability to the unknown parameter!

In [ ]:
def prior(lam):
    if lam < 0 or lam > 10:
        return 0.0
    return 1.0

def likelihood(k, lam):
    return st.poisson(lam).pmf(k)

def posterior(lam, k):
    return likelihood(k, lam) * prior(lam)

lambdas = np.linspace(0.0, 20, 1001)
likelihoods = st.poisson.pmf(counts, lambdas)
posteriors = np.array([posterior(lam, counts) for lam in lambdas])

normalized_posteriors = posteriors / np.trapz(posteriors, lambdas)


plt.figure()
plt.plot(lambdas, likelihoods, color="b", ls="-", lw=4, alpha=0.5, label="Likelihood")
plt.plot(lambdas, posteriors, "r-", label="Unnormalized posterior")
plt.plot(lambdas, normalized_posteriors, "k:", label="Normalized posterior")
plt.xlabel("$\lambda$")
plt.ylabel("Probability density")
plt.legend(loc="upper right")
plt.show()

In [ ]:
def calculate_mode_mean_and_std(x, Px):
    mode = x[np.argmax(Px)]
    mean = sum(x * Px) / sum(Px)
    variance = sum((x - mean) ** 2.0 * Px) / sum(Px)
    return mode, mean, variance ** 0.5

mode, mean, std = calculate_mode_mean_and_std(lambdas, likelihoods)
print(f"(using only the likelihood)   mode = {mode:.2f} | mean +/- std = {mean:.2f} +/- {std:.2f}")
mode, mean, std = calculate_mode_mean_and_std(lambdas, posteriors)
print(f"(unnormalized posterior)      mode = {mode:.2f} | mean +/- std = {mean:.2f} +/- {std:.2f}")
mode, mean, std = calculate_mode_mean_and_std(lambdas, normalized_posteriors)
print(f"(normalized   posterior)      mode = {mode:.2f} | mean +/- std = {mean:.2f} +/- {std:.2f}")

Notes:
> The prior alters the shape of the posterior and makes it different from the likelihood. A "strict" or informative prior will dominate the final result, while a "loose" or uniformative prior will have negligible effect.
> Using a uniform prior possibly affects the bounds and the normalization, but the maximum a posteriori values might be the same as the maximum likelihood.

### Questions for discussion

#### Q1. Is the mean value of $\lambda$ the same as if we only used the likelihood?

#### Q2. Is the uncertainty affected by the prior?

#### Q3. Does the normalization of the posterior affect the parameter estimation? 

# 4. Measuring a source luminosity in the presence of background

In our X-ray image we see that there a detected photons all over the place. These may come from the detector (instrumental background) or foreground (stars in Mikly Way) or background (distant quasars) sources. If we are interested in the luminosity of a source in NGC 1482, we want to correct for the "background". Previous studies have measured the flux or photon counts per unit area, in the bandwidth of our observation. The source detection algorithm that gave as the 5 counts, also found the "source region" and based on its area, we find that we expect on average $\beta=3$ counts from the background!

In [ ]:
beta = 3

## 4.1. How many are the source counts $n$ if we measured $k$ counts and expect $\beta$ counts?

### Method 1: Simply substract the background and assume Poisson distribution

In [ ]:
n = counts - beta
n_err = n ** 0.5
print(f"Source counts: {n:.2f} +/- {n_err:.2f}")

#### Q4. What if the background level was higher than the detected counts? Are the source counts definitely 0? If not, is there an uncertainty?

### Method 2: The mean background, is a... mean. Take into account the Poisson nature of it...

In [ ]:
counts_err = 0.0   # the counts we got are fixed
beta_err = beta ** 0.5

n = counts - beta
n_err = (counts_err ** 2.0 + beta_err ** 2.0) ** 0.5
print(f"Source counts: {n:.2f} +/- {n_err:.2f}")

Now the probability distribution of the source counts is not Poisson. The uncertainty comes from applying the one-term approximation of the error-propagation formula (which furthermore assumes Gaussian errors).

### Method 3: A "combinatorial" approach

The probability of having no counts from the source, is the probability that they all detected counts come from the background. The probability of one source count = 4 counts from the background, and so on.

$$\large P(n | k , \beta) = \text{Pois}(k - n; \beta) $$

In [ ]:
beta = 3.0
n = np.arange(0, 11)
Pn = st.poisson(beta).pmf(counts - n)
# now normalize it...
Pn /= sum(Pn)

plt.figure()
plt.bar(n, Pn, color="r")
plt.ylabel("Probability")
plt.xlabel("Number of source counts")
plt.show()

In [ ]:
mode, mean, std = calculate_mode_mean_and_std(n, Pn)
print(f"mode = {mode:.2f} | mean +/- std = {mean:.2f} +/- {std:.2f}")

#### Q6. What is the problem with this approach? 
By accounting for the background (and its Poissonian nature) we are able to quantify the likelihood of a given number of counts coming from a source. However, we never took into account the Poissonian nature of the source counts! We completely exclude the case we happened to not get any count from a powerful source (which would typically give back, say, 6 counts).

## 4.2. The Bayesian approach


### Writing what we know about the data, nature and their connection...

We detect $k$ counts, out of which $n$ are coming from a source with $\lambda$ counts on average, and $b$ counts from the background with mean level $\beta$. Both the souce and the background counts are Poisson distributed:

$$ \large
\begin{align}
k &= n + b \\
n &\sim \text{Pois}(\lambda) \\
b &\sim \text{Pois}(\beta)
\end{align}
$$

### Writing things we want to know (left side) as a function of what we do know (right side)...

Given that we have a measurement of $k$ and $\beta$ (from a previous study), we want to know the $\lambda$. We use the Bayes' rule (neglecting evidence) to quantify the posterior on $\lambda$:

$$\large P(\lambda | k, \beta) \propto P(k | \lambda, \beta) P(\lambda | \beta) $$

The **likelihood** requires the knowledge of what is the number of total counts given the mean source and backgroud counts. In this case we are lucky: the sum of Poisson random variables is a new Poisson variable with mean value equal to the sum of the mean values of the original Poisson variables:

$$ \large k \sim \text{Pois}(\lambda + \beta) $$

This makes complete sense: the Universe doesn't care about what we define as a source or background! For all we know we can consider a source as two objects: the first and second halves!
Therefore,

$$ \large P(k | \lambda, \beta) = \text{Pois}(k; \lambda + \beta) = \text{Pois}(k; \lambda + 3)$$

(the last part comes from the fact that we assume infinite accuracy on $\beta$.)

For the **prior** on $\lambda$ we select the improper prior that allows only non-negative values, and therefore

$$\large 
P(\lambda | k, \beta) \propto 
    \begin{cases} 
        \text{Pois}(k; \lambda + 3), &\lambda \geq 0 \\
        0, &\lambda < 0 
    \end{cases}
$$

## Exercise 4: Construct the likelihood, prior and posterior functions

In [ ]:
def likelihood(total_counts, source_mean, bkg_mean):
    return st.poisson(...).pmf(total_counts)

def prior(source_mean):
    return ... if source_mean < 0 else ...

def posterior(source_mean, total_counts, bkg_mean):
    return likelihood(total_counts=total_counts, source_mean=source_mean, bkg_mean=bkg_mean) * prior(source_mean)

lambdas = np.linspace(0.0, 20.0, 101)
posteriors_fixedbkg = np.array([posterior(source_mean=lam, total_counts=counts, bkg_mean=beta) for lam in lambdas])

plt.figure()
plt.plot(lambdas, posteriors_fixedbkg, "k-")
plt.xlabel("Source mean counts")
plt.ylabel("Posterior density")
plt.show()

In [ ]:
mode, mean, std = calculate_mode_mean_and_std(lambdas, posteriors_fixedbkg)
print(f"COUNTS    :       mode = {mode:8.2f} | mean +/- std = {mean:8.2f} +/- {std:8.2f}")

cdf = np.cumsum(posteriors_fixedbkg) / np.sum(posteriors_fixedbkg)

# mode, mean, std = calculate_mode_mean_and_std(lambdas, posteriors_fixedbkg)
index_p16 = np.argmin(np.abs(cdf-0.16))
index_p84 = np.argmin(np.abs(cdf-0.84))
index_p025 = np.argmin(np.abs(cdf-0.025))
index_p975 = np.argmin(np.abs(cdf-0.975))
index_p001 = np.argmin(np.abs(cdf-0.0005))
index_p999 = np.argmin(np.abs(cdf-0.9995))

factor = flux_to_luminosity * counts_to_flux_factor / exposure_time_in_seconds

print(f"LUMINOSITY:       mode = {factor*mode:8.2e} | mean +/- std = {factor*mean:8.2e} +/- {factor*std:8.2e}")
print(f"               68%   CI: ({factor*lambdas[index_p16]:.2e}, {factor*lambdas[index_p84]:.2e})")
print(f"               95%   CI: ({factor*lambdas[index_p025]:.2e}, {factor*lambdas[index_p975]:.2e})")
print(f"               99.9% CI: ({factor*lambdas[index_p001]:.2e}, {factor*lambdas[index_p999]:.2e})")

## Observations

* Using the Bayesian interpretation of probability we quantified the likelihood of having a source of a given intensity.
* We took into account all information: Poissonian nature of both source and background counts
* We were allowed to apply constraints from previous studies (in the prior)
* The result looks as expected: it is defined only for non-negative values, peaks close to 0 as we would expect from a weak detection, but does not exclude a powerful source (no clear upper limit)!
* We can use the well-defined posterior to report using point estimates (e.g., mean, mode, std), confidence intervals (or regions), etc.

# 4.3. Using uncertainties in the background

The mean background level of $3$ was just an example of what the literature might report. In reality, researchers put uncertainties in that value as well! Let's assume it's $3 \pm 1.0$.

What is the posterior now? The background level $\beta$ cannot be treated as background information. It is not a fixed value, but a parameter for which we do not have absolute knowledge. Consequently, we have to consider all posibilities for the mean source and background counts.

$$\large P(\lambda, \beta | k) \propto P(k | \lambda, \beta) P(\lambda, \beta)$$

The likelihood is still as in the previous example:

$$ \large P(k | \lambda, \beta) = \text{Pois}(k; \lambda + \beta)$$

The *joint prior* can be broken into simpler terms:

$$\large P(\lambda, \beta) = P(\lambda | \beta) P(\beta)$$


(*Note that we could write it as $P(\lambda, \beta) = P(\beta | \lambda) P(\lambda) $, but the following analysis doesn't change!*)

The first term is quantifying the prior on $\lambda$ but given the value of $\beta$, taking into account any possible correlation between our parameters. In our case, we have no reason to believe they are correlated: what a source in our galaxy does is unrelated to the emission from sources in the Milky Way (foreground) or the distant Universe (background). Consequently, $P(\lambda | \beta) = P(\lambda)$, and:

$$\large P(\lambda, \beta) = P(\lambda) P(\beta) $$

We can use the same prior on $\lambda$ as before (uniform for $\lambda \geq 0$). For the $\beta$ we use the literature values to take a truncated version of the normal distribution (background is non-negative):

$$\large P(\beta) \propto \text{Norm}(\beta; \mu=3, \sigma=1) \quad \text{for} \quad b \geq 0 $$

Consequently, the two-dimensional posterior on both $\lambda$ and $\beta$ is

$$\large 
P(\lambda, \beta | k) \propto 
    \begin{cases} 
        \text{Pois}(k; \lambda + \beta) \times \text{Norm}(\beta; \mu=3, \sigma=1), &\lambda, \beta \geq 0 \\
        0, &\lambda < 0 \text{ or } b < 0
    \end{cases}
$$

The background level $\beta$ might be unknown but we are not specifically interested in its value. We only care about the source level $\lambda$. A quantity that we are not interested in but has to enter the Bayesian analysis is a **nuisance parameter**. To infer the quantities of interest, we use marginalization by integrating over all possible values for the nuisance parameter (or summation when it is discrete):

$$\large 
P(\lambda | k) = \int_{-\infty}^{\infty} P(\lambda, \beta | k) d\beta
$$

Taking into acount the limits ($b$ is non-negative)...

$$\large 
P(\lambda | k) = \int_{0}^{\infty} P(\lambda, \beta | k) d\beta
$$

In [ ]:
import scipy.integrate

beta = 3.0
beta_err = 1.0

def likelihood(total_counts, source_mean, bkg_mean):
    return st.poisson.pmf(total_counts, source_mean + bkg_mean)

def prior_source(source_mean):
    return 0.0 if source_mean < 0 else 1.0

def prior_background(bkg_mean):
    return st.norm.pdf(bkg_mean, beta, beta_err)

def joint_posterior(source_mean, total_counts, bkg_mean):
    return (likelihood(total_counts=total_counts, source_mean=source_mean, bkg_mean=bkg_mean) 
            * prior_source(source_mean) * prior_background(bkg_mean))

def posterior(source_mean, total_counts):
    lower_limit = max(0, beta - beta_err*5)
    upper_limit = beta + beta_err*5
    
    def integrand(background):
        return joint_posterior(source_mean=source_mean, 
                               total_counts=total_counts, 
                               bkg_mean=background)

    result = scipy.integrate.quad(integrand, a=lower_limit, b=upper_limit)
    return result[0]

posteriors_varbkg = np.array([posterior(source_mean=lam, total_counts=counts) for lam in lambdas])

plt.figure()
plt.plot(lambdas, posteriors_fixedbkg / np.trapz(posteriors_fixedbkg, x=lambdas), "k-", label="Fixed background")
plt.plot(lambdas, posteriors_varbkg / np.trapz(posteriors_varbkg, x=lambdas), "r--", label="Background has a distribution")
plt.xlabel("Source mean counts")
plt.ylabel("Posterior density")
plt.legend(loc="upper right")
plt.show()

In [ ]:
mode, mean, std = calculate_mode_mean_and_std(lambdas, posteriors_fixedbkg)
print(f"FIXED BACKGROUND:       mode = {mode:.2f} | mean +/- std = {mean:.2f} +/- {std:.2f}")
mode, mean, std = calculate_mode_mean_and_std(lambdas, posteriors_varbkg)
print(f"VARIABLE BACKGROUND:    mode = {mode:.2f} | mean +/- std = {mean:.2f} +/- {std:.2f}")